# Image Classification -- Understanding the Process

Code taken and adapted from https://debuggercafe.com/visualizing-filters-and-feature-maps-in-convolutional-neural-networks-using-pytorch/ and 

In [ ]:
import torch
import torch.nn as nn
from torchvision import models # PyTorch image processing lib
from tqdm.notebook import trange, tqdm

## Load and Preprocess an image

In [ ]:
import cv2 as cv # python library for image processing
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
raw = cv.imread("data/annot3_face2.png") # f"data/images/space_shuttle.jpg")# 
raw = cv.cvtColor(raw, cv.COLOR_BGR2RGB) # convert colorspace
plt.imshow(raw)
plt.show()

## umformen

In [ ]:
pil_transform = transforms.Compose([
    transforms.ToPILImage(), # Python Imaging Library
    transforms.Resize(224),
    transforms.CenterCrop(224),
    ])    

# adapt normalization from https://pytorch.org/vision/master/models.html
normalize_transform = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])     

preprocess_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize_transform
    ])    

#added
test_transforms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


In [ ]:
print("Original image shape: ", raw.shape)
plt.imshow(raw)

In [ ]:
np_img = np.array(raw)
pil_img = pil_transform(np_img)

print("PIL image is in mode: ", pil_img.mode, "with ", len(pil_img.split()), "channels and size:", pil_img.size)
display(pil_img)

In [ ]:
tensor_img = preprocess_transform(pil_img)
print(tensor_img.size())
# unsqueeze to add a batch dimension
#img = img.unsqueeze(0)
print(tensor_img.size())
plt.imshow(tensor_img.transpose(2,0).transpose(0,1))

## Predict the image

In [ ]:
# added:
from model import model_static
model = model_static("data/model_weights.pkl")
model_dict = model.state_dict()
snapshot = torch.load("data/model_weights.pkl")
model_dict.update(snapshot)
model.load_state_dict(model_dict)

model.cuda()
#model.to(torch.device('cuda:1'))
model.train(False);

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


In [ ]:
out=model(tensor_img.unsqueeze_(0).cuda())

# added
score = torch.sigmoid(out)
print(score)

We get 1000 raw scores. One per class.

In [ ]:
_, index = torch.max(out, 1)
index

## Find an Explanation for the Classification

For the Lime package install ````lime````

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
def batch_predict(images):
    model.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)
    out = model(batch.cuda())
    return torch.sigmoid(out).cpu().detach().numpy()

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(pil_transform(np_img)), 
                                         batch_predict, # classification function
                                         top_labels=5, 
                                         num_samples=1000) # number of images that will be sent to classification function

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry1 = mark_boundaries(temp/255.0, mask)
print(type(img_boundry1))
print(img_boundry1.shape)
plt.imshow(img_boundry1);
#plt.savefig("data/images/annot3bild2.jpg")